In [1]:
from __future__ import print_function
from scipy.misc import imsave
import numpy as np
import time
import os
import h5py

#import os #change theano device to cpu (it is gpu as default)    
#os.environ['THEANO_FLAGS'] = "device=cpu"    
import theano
import sklearn

Using gpu device 0: GeForce GTX 960 (CNMeM is disabled, cuDNN not available)


In [2]:
theano.config.device

'gpu'

In [3]:
# load data
data = np.genfromtxt('train.csv', delimiter=',')
data = data[1:,:] # ignore header

X_final = np.genfromtxt('test.csv', delimiter=',')
X_final = X_final[1:,:]

In [10]:
#split data into train, CV, test

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils


from keras.utils import np_utils
from sklearn.cross_validation import train_test_split

train_set, test_set = train_test_split(data, test_size=0.50, random_state=42)
train_set, val_set = train_test_split(train_set, test_size=0.30, random_state=42)

print ('Training set   : %d data points' % len(train_set))
print ('Validation set : %d data points' % len(val_set))
print ('Test set       : %d data points' % len(test_set))


batch_size = 128
nb_classes = 10
nb_epoch = 2

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
nb_pool = 2
# convolution kernel size
kernel_size = (3, 3)
               
# split into input (X) and output (Y) variables
y_train = train_set[:,0]
X_train = train_set[:,1:]

y_val = val_set[:,0]
X_val = val_set[:,1:]

y_test = test_set[:,0]
X_test = test_set[:,1:]

y_all = data[:,0]
X_all = data[:,1:]

img_rows = 28
img_cols = 28
X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_final = X_final.reshape(X_final.shape[0], 1, img_rows, img_cols)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_final = X_final.astype('float32')

X_train /= 255
X_test /= 255
X_final /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print(X_final.shape[0], 'final samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


Training set   : 14700 data points
Validation set : 6300 data points
Test set       : 21000 data points
X_train shape: (14700L, 1L, 28L, 28L)
14700 train samples
21000 test samples
28000 final samples


In [11]:
print (y_train[0])
print (Y_train[0])

6.0
[ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]


In [12]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(nb_pool, nb_pool)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 14700 samples, validate on 21000 samples
Epoch 1/2
14700/14700 [==============================] - 9s - loss: 0.8337 - acc: 0.7361 - val_loss: 0.2773 - val_acc: 0.9175
Epoch 2/2
14700/14700 [==============================] - 9s - loss: 0.3230 - acc: 0.9021 - val_loss: 0.1817 - val_acc: 0.9458
Test score: 0.18168841905
Test accuracy: 0.94580952381


In [12]:
y_final = model.predict(X_final, verbose=False)

predictions = np.argmax(y_final, axis=1)
predictions = np.reshape(predictions,[predictions.size, 1])

img_id = np.linspace(1,predictions.shape[0],predictions.shape[0])
img_id = np.reshape(img_id, [img_id.shape[0], 1])

img_id_and_predictions = np.append(img_id, predictions, axis=1)

np.savetxt("my_submission5.csv", img_id_and_predictions, delimiter=',',fmt='%.0f', header= 'ImageId,Label', comments='' )

## Let's try to get these weights and filters

In [30]:
weights=model.layers[0].get_weights()
weights[0].shape

(32L, 1L, 3L, 3L)

In [21]:
model2 = Sequential()

model2.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=(1, img_rows, img_cols), 
                        weights=model.layers[0].get_weights()))
model2.add(Activation('relu'))
#model2.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
#model2.add(Activation('relu'))

activations = model2.predict(X_train)

In [22]:
print(activations.shape)


(14700L, 32L, 26L, 26L)


In [23]:
X_train.shape

(14700L, 1L, 28L, 28L)

In [24]:
a= activations[0,5,:,:]
a /= a.max() 
import matplotlib.pyplot as plt
axes = plt.gca()
axes.set_xlim([0,26])
axes.set_ylim([0,26])
plt.imshow(a)
plt.colorbar(orientation='vertical')
plt.show()

In [25]:
a.shape

(26L, 26L)

In [26]:
X_sample = X_train[0,0,:,:]
X_sample.shape

(28L, 28L)

In [27]:
(activations[0,0,:,:]).max()

0.12101162

## figure(1) shows ouput of first layer with image #1000 as input
## figure(2) shows image #1000



In [28]:
import matplotlib.pyplot as plt

j = 1000
plt.figure(1)
axes.set_xlim([0,25])
axes.set_ylim([0,25])

for i in xrange(32):
    plt.subplot(8,4,i+1)
    plt.axis('off')
    plt.imshow(activations[j,i,:,:]/(activations[j,i,:,:]).max())

plt.tight_layout(pad=0, w_pad=0, h_pad=0)
plt.show()

plt.figure(2)
axes.set_xlim([0,27])
axes.set_ylim([0,27])
plt.imshow(X_train[j,0,:,:]/(X_train[j,0,:,:]).max())
plt.show()